# **`AI-Powered Story Generation Using Gemini-1.5-Flash`**

## Set Up Google Gemini API Access

1. **Get Access to Google Gemini API**  
   - Sign up for Gemini API access at [https://ai.google.dev/](https://ai.google.dev/).
   - Obtain your Gemini API key from the Google AI Studio dashboard.

2. **Configure Your API Key**  
   - Create a file named `config.py` in the same folder as your notebook.
   - Add your API key to `config.py` as follows:
     ```python
     API_KEY = "your_actual_gemini_api_key_here"
     ```

3. **Install Required Python Packages**  
   In your notebook or terminal, run:
   ```python
   !pip install google-generativeai textstat pandas numpy
   ```

> **Note:** Keep your API key secure and do not share it publicly.  
> You must have your API key configured before running any Gemini model code in the

# Installing Required Libraries

In [ ]:
!pip install -q google-generativeai textstat pandas

In [1]:
# Import necessary libraries
import google.generativeai as genai # Google Gemini API
import pandas as pd # Data handling and analysis
import textstat # Text statistics and readability metrics
from time import time # Time tracking for performance
import re # Regular expressions for text processing
import config # Configuration file for API key


d:\360 DigiTMG projects\Project-4\AI-Powered Story Generation and Narration System\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **`API Setup`**

In [2]:
# Replace this with your actual Gemini API Key
api_key = config.API_KEY

genai.configure(api_key=api_key)

# Load Gemini Flash model
model = genai.GenerativeModel("gemini-1.5-flash")



# **`Story Generation Function`**

In [19]:
def generate_story_with_title(age_range, genre, characters, theme):
    import re
    from time import time

    flesch_map = {
        "3-8": (80, 100),
        "9-15": (60, 80),
        "16-19": (50, 60),
        "20+": (30, 50),
    }

    def get_flesch_band(age_range):
        age_range = age_range.replace(" ", "")
        if "-" in age_range:
            start, end = map(int, age_range.split("-"))
            if 3 <= start <= 5 and end <= 8:
                return 80, 100
            elif 9 <= start and end <= 15:
                return 60, 80
            elif 16 <= start and end <= 19:
                return 50, 60
        elif "+" in age_range:
            return 30, 50
        raise ValueError("Invalid or unsupported age range.")

    score_min, score_max = get_flesch_band(age_range)
    flesch_score = f"{score_min}–{score_max}"

    prompt = f"""
    Write an imaginative and age-appropriate story for Indian children aged {age_range}.

    Requirements:
    - Genre: {genre}
    - Main Characters: {characters}
    - Theme: {theme}
    - Story Length: Maximum 350 words.
    - Use simple, clear English vocabulary and sentence structure that is suitable for children aged {age_range}.
    - The story MUST be written so that its Flesch Reading Ease (FRE) score is between {flesch_score}. 
    - The Flesch score requirement applies ONLY to the English story.
    - DO NOT include any translation, non-English words, or translation labels in the English story section.
    - Avoid complex words and long sentences for younger ages; use more advanced language for older ages.
    - Include a catchy, relevant title at the beginning.
    - End with a moral in the format: Moral: [your moral]
    - **Do NOT add any label like 'English Story:' or similar. Only use 'Title:', the story text, and 'Moral:' as shown below.**

    Format:
    Title: [Your title]
    [Story text]
    Moral: [your moral]
    """

    # Append age-specific FRE instructions
    if age_range in ["3-8"]:
        prompt += (
            "\n- IMPORTANT: The story MUST have a Flesch Reading Ease (FRE) score between 80 and 100."
            "\n- Use normal words and keep medium sentences (6-9 words each)."
            "\n- Avoid any very easy or very complex vocabulary."
            "\n- Imagine you are writing for a 3–8 year old who is just learning to read."
            "\n- If the story is very easy or more difficult, REWRITE it until it fits the FRE score range."
            "\n- Do NOT write a story that is outside this FRE score range."
            "\n- If you cannot write a story within this FRE range, DO NOT RETURN ANY STORY."
        )
    elif age_range in ["9-15"]:
        prompt += (
            "\n- IMPORTANT: The story MUST have a Flesch Reading Ease (FRE) score between 60 and 80."
            "\n- The FRE score must NEVER be above 80 or below 60 for this age group."
            "\n- Use simple and clear words."
            "\n- Keep sentences medium (10–14 words) and normal to understand."
            "\n- Avoid very difficult vocabulary and long sentences."
            "\n- Do not use advanced or academic words."
            "\n- Imagine you are writing for a school student aged 9 to 15."
            "\n- If the story is too easy or too hard, or if the FRE score is outside 60–80, REWRITE it."
            "\n- If you cannot write a story within this FRE range, DO NOT RETURN ANY STORY."
        )
    elif age_range in ["16-19"]:
        prompt += (
            "\n- IMPORTANT: The story MUST have a Flesch Reading Ease (FRE) score between 50 and 60."
            "\n- Use clear language with some easy vocabulary."
            "\n- Keep most sentences between 10 and 16 words."
            "\n- Mix simple and moderate sentences, but avoid very long or academic sentences."
            "\n- Write as you would for a high school student aged 16 to 19."
            "\n- If the story is too easy or too hard, REWRITE it."
            "\n- If you cannot write a story within this FRE range, DO NOT RETURN ANY STORY."
        )
    elif age_range in ["20+"]:
        prompt += (
            "\n- IMPORTANT: The story MUST have a Flesch Reading Ease (FRE) score between 30 and 50."
            "\n- Aim for a FRE score between 40 and 45. Do NOT write a story with a FRE score below 35 or above 45."
            "\n- Use advanced vocabulary, longer sentences, and more complex sentence structures."
            "\n- Write as you would for college students or adults."
            "\n- If the story is too easy (FRE > 50) or too hard (FRE < 30), REWRITE it."
            "\n- If you cannot write a story within this FRE range, DO NOT RETURN ANY STORY."
        )

    try:
        start = time()
        response = model.generate_content(prompt)
        latency = round(time() - start, 2)
        full_text = response.text.strip()

        match = re.search(r"Title:\s*(.*)", full_text)
        if match:
            title = match.group(1).strip()
            story = full_text.replace(match.group(0), "").strip()
        else:
            title = full_text.split("\n")[0].strip()
            story = "\n".join(full_text.split("\n")[1:]).strip()

        return title, story, latency
    except Exception as e:
        return "Error", f"Error: {e}", 0


# **`Evaluation`**

In [8]:
# Function to evaluate the story for readability and other metrics
def evaluate_story(text):
    story_only = text.split("Story:")[-1]
    return {
        "Flesch Score": textstat.flesch_reading_ease(story_only),
        "Grade Level": textstat.flesch_kincaid_grade(story_only),
        "Word Count": textstat.lexicon_count(story_only),
        "Sentence Count": textstat.sentence_count(story_only),
        "Character Count": len(story_only)
    }


In [13]:
# Function to evaluate the Flesch Reading Ease score of a given text
def evaluate_flesch(text):
    return round(textstat.flesch_reading_ease(text), 2)


# **`Run a sample story`**

In [15]:
# Example usage of the story generation function
title, story, latency = generate_story_with_title(
    age_range="20+",
    genre="Friendhip",
    characters="Divya and Daya",
    theme="Humanity and Kindness"
)

print(f" Title: {title}\n")
print(f" Story:\n{story}\n")
print(f" Generation Time: {latency} sec")
print(f" Flesch Score: {evaluate_flesch(story)}")


 Title: The Mango Tree and the Shared Feast

 Story:
Divya, a spirited artist with a penchant for vibrant colours, and Daya, a quiet scholar with a love for ancient texts, were an unlikely pair.  Their friendship, forged in the shared shade of a sprawling mango tree on the university campus, blossomed unexpectedly.  Divya, impulsive and generous, often shared her vibrant paintings, while Daya, contemplative and insightful, offered profound interpretations of their shared experiences.  One sweltering afternoon, a group of homeless children huddled near their tree, their hunger palpable.  Divya, moved by their plight, immediately decided to share her lunch – a delicious spread of homemade treats her mother had packed. Daya, initially hesitant, understood the depth of Divya's compassion.  He quietly added his own simple meal, a few chapatis and some pickle.  The children, initially shy, devoured the food with grateful smiles.  The shared meal transcended their different backgrounds, foste

In [20]:

# Define Flesch targets mapping
flesch_targets = {
    "3-8": "80–100",
    "9-15": "60–80",
    "16-19": "50–60",
    "20+": "30–50",
}
# Function to get the Flesch target based on age range
def get_flesch_target(age_range):
    age_range = age_range.replace(" ", "")
    if "-" in age_range:
        start, end = [int(x) for x in age_range.split("-")]
        if 3 <= start and end <= 8:
            return flesch_targets["3-8"]
        elif 9 <= start and end <= 15:
            return flesch_targets["9-15"]
        elif 16 <= start and end <= 19:
            return flesch_targets["16-19"]
        elif start >= 20 or end >= 20:
            return flesch_targets["20+"]
    elif "+" in age_range:
        return flesch_targets["20+"]
    return "N/A"


# Example inputs for generating multiple stories
inputs = [
    ("3-8", "Fantasy", "a kind dragon and a brave rabbit", "kindness"),
    ("9-15", "Adventure", "a clever squirrel and a helpful bird", "teamwork"),
    ("16-19", "Sci-Fi", "a boy and his time-traveling robot", "curiosity"),
    ("20+", "Animal Tale", "a turtle and a butterfly", "patience"),
    ("3-8", "Magical Tale", "a singing frog and a dancing fairy", "friendship")
]
# Generate stories for each input and evaluate their Flesch scores
results = []
# Iterate through each input and generate a story
for age, genre, chars, theme in inputs:
    title, story, latency = generate_story_with_title(age, genre, chars, theme)
    flesch = evaluate_flesch(story)
    flesch_target = get_flesch_target(age)
    results.append({
        "Age Range": age,
        "Flesch Target": flesch_target,
        "Genre": genre,
        "Characters": chars,
        "Theme": theme,
        "Title": title,
        "Latency (sec)": latency,
        "Flesch Reading Score": flesch,
        "Story": story
    })

# Create a DataFrame to hold the results
df = pd.DataFrame(results)
# Function to calculate the Flesch error based on the target range
def flesch_error_within_range(predicted, flesch_target):
    if "–" in flesch_target:
        low, high = flesch_target.split("–")
        try:
            low = float(low)
            high = float(high)
            if low <= predicted <= high:
                return 0
            elif predicted < low:
                return predicted - low
            else:  # predicted > high
                return predicted - high
        except:
            return None
    return None

# Calculate Flesch Error for each story
df["Flesch Error"] = df.apply(
    lambda row: flesch_error_within_range(row["Flesch Reading Score"], row["Flesch Target"]),
    axis=1
)

#  Calculate Averages
avg_latency = round(df["Latency (sec)"].mean(), 2) # Average story generation time
avg_flesch = round(df["Flesch Reading Score"].mean(), 2) # Average Flesch Reading Score

# Print the results 
print("5 Stories Generated\n") # 5 stories generated
print(df[["Title", "Age Range", "Flesch Target", "Latency (sec)", "Flesch Reading Score", 'Flesch Error']])

print("\n Averages:")
print(f" Average Story Generation Time: {avg_latency} sec") # Average story generation time
print(f" Average Flesch Reading Score: {avg_flesch}") # Average Flesch Reading Score

# Calculate RMSE of Flesch Error
import numpy as np # Numerical operations
rmse = np.sqrt(np.mean(df["Flesch Error"] ** 2)) # Root Mean Square Error of Flesch Error
print(f"RMSE of Flesch Error: {rmse:.2f}")



5 Stories Generated

                                    Title Age Range Flesch Target  \
0     Raja the Dragon and Rani the Rabbit       3-8        80–100   
1                 The Great Mango Mystery      9-15         60–80   
2      Rohan and Chrono's Curious Journey     16-19         50–60   
3  The Tortoise and the Monarch's Pursuit       20+         30–50   
4  The Singing Frog and the Dancing Fairy       3-8        80–100   

   Latency (sec)  Flesch Reading Score  Flesch Error  
0           2.09                 82.71          0.00  
1           2.19                 76.38          0.00  
2           2.76                 47.45         -2.55  
3           2.35                 44.02          0.00  
4           1.94                 88.27          0.00  

 Averages:
 Average Story Generation Time: 2.27 sec
 Average Flesch Reading Score: 67.77
RMSE of Flesch Error: 1.14


In [31]:
import numpy as np

rmse = np.sqrt(np.mean(df["Flesch Error"] ** 2))
print(f"RMSE of Flesch Error: {rmse:.2f}")

RMSE of Flesch Error: 3.85


#  **`Flesch Score`**

The Flesch Score tells you how easy a text is to read.

| **Age Range** | **Label**         | **Target Flesch Score** | **Reading Difficulty** |
|---------------|-------------------|-------------------------|------------------------|
| **3–8**       | Early Readers     | **80–100**              | Very Easy              |
| **9–15**      | Pre-teens/Teens   | **60–80**               | Easy to Fairly Easy    |
| **16–19**     | Older Teens       | **50–60**               | Fairly Difficult       |
| **20+**       | Adults            | **30–50**               | Difficult              |





# Summary of `Gemini_1.5_Flash.ipynb`

## Purpose
This notebook demonstrates how to use the Google Gemini-1.5-Flash API to generate age-appropriate, readable stories for children and adults, focusing on Indian contexts.

---

## Key Sections & Functionality

### 1. Setup & Installation
- Instructions for obtaining and configuring a Gemini API key.
- Installs required libraries: `google-generativeai`, `textstat`, `pandas`, `numpy`.

### 2. API Initialization
- Loads the Gemini-1.5-Flash model using the API key from a `config.py` file.

### 3. Story Generation
- `generate_story_with_title()` function:
  - Accepts age range, genre, characters, and theme.
  - Crafts a prompt with Flesch Reading Ease (FRE) targets based on age.
  - Calls Gemini API to generate a story with a title and moral.
  - Returns the title, story, and generation latency.

### 4. Evaluation
- Functions to evaluate readability (`evaluate_story`, `evaluate_flesch`) using `textstat`.
- Calculates Flesch Reading Ease, grade level, word/sentence/character counts.

### 5. Batch Story Generation & Analysis
- Generates multiple stories for different age groups and genres.
- Collects results in a DataFrame.
- Calculates Flesch score errors, averages, and RMSE (Root Mean Square Error) for Flesch scores.

### 6. Flesch Score Reference Table
- Provides a table mapping age ranges to target Flesch scores and reading difficulty.

---

## Usage
- The notebook is ready for both single and batch story generation.
- It evaluates how well generated stories match the target readability for each age group.

---

**In summary:**  
This notebook is a complete workflow for generating and evaluating children’s stories using the Gemini-1.5-Flash API, with a strong emphasis on age-appropriate